In [20]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.contrib.keras as keras
import sys
import utils

## Data Preprocessing

First, we load in our data set with the `utils` helpers that are provided along with the dataset that can be found [here](https://github.com/mdeff/fma).

In [14]:
tracks = utils.load('./fma_metadata/tracks.csv', nrows=100)
features = utils.load('./fma_metadata/features.csv', nrows=100)

/Users/rockzhou/workstation/ml/music_analysis/utils.py:37: FutureWarning: specifying 'categories' or 'ordered' in .astype() is deprecated; pass a CategoricalDtype instead
  'category', categories=SUBSETS, ordered=True)


Then, we filter down the tracks DataFrame to only the `track_id` and `genres`.

In [32]:
# dataframe with only track_id and genre ids
trackid_genres = tracks['track'].filter(['track_id', 'genres'])

For simplicity, we only consider the first genre listed for each song.

In [33]:
# only take the first genre if a song has multiple genres
def getFirstGenre(genres):
    return genres[0]

reduceGenres = np.vectorize(getFirstGenre)
y_total = reduceGenres(trackid_genres.values)

Then, we convert our `y` vector into one hot encoding using `keras`:

In [31]:
y_total_onehot = keras.utils.to_categorical(y_total)

In [30]:
X_total = pd.merge(
    trackid_genres.head(100),
    features.head(100),
    on='track_id'
).drop('genres', axis=1)

,"(chroma_cens, kurtosis, 01)","(chroma_cens, kurtosis, 02)","(chroma_cens, kurtosis, 03)","(chroma_cens, kurtosis, 04)","(chroma_cens, kurtosis, 05)","(chroma_cens, kurtosis, 06)","(chroma_cens, kurtosis, 07)","(chroma_cens, kurtosis, 08)","(chroma_cens, kurtosis, 09)","(chroma_cens, kurtosis, 10)",...,"(tonnetz, std, 04)","(tonnetz, std, 05)","(tonnetz, std, 06)","(zcr, kurtosis, 01)","(zcr, max, 01)","(zcr, mean, 01)","(zcr, median, 01)","(zcr, min, 01)","(zcr, skew, 01)","(zcr, std, 01)"
track_id,,,,,,,,,,,,,,,,,,,,,
2,7.180653,5.230309,0.249321,1.347620,1.482478,0.531371,1.481593,2.691455,0.866868,1.341231,...,0.054125,0.012226,0.012111,5.758890,0.459473,0.085629,0.071289,0.000000,2.089872,0.061448
3,1.888963,0.760539,0.345297,2.295201,1.654031,0.067592,1.366848,1.054094,0.108103,0.619185,...,0.063831,0.014212,0.017740,2.824694,0.466309,0.084578,0.063965,0.000000,1.716724,0.069330
5,0.527563,-0.077654,-0.279610,0.685883,1.937570,0.880839,-0.923192,-0.927232,0.666617,1.038546,...,0.040730,0.012691,0.014759,6.808415,0.375000,0.053114,0.041504,0.000000,2.193303,0.044861
10,3.702245,-0.291193,2.196742,-0.234449,1.367364,0.998411,1.770694,1.604566,0.521217,1.982386,...,0.074358,0.017952,0.013921,21.434212,0.452148,0.077515,0.071777,0.000000,3.542325,0.040800
20,-0.193837,-0.198527,0.201546,0.258556,0.775204,0.084794,-0.289294,-0.816410,0.043851,-0.804761,...,0.095003,0.022492,0.021355,16.669037,0.469727,0.047225,0.040039,0.000977,3.189831,0.030993


Finally, we save our processed data as `csv` files on the filesystem.